In [ ]:
# %%writefile galapagosun_bwd.py

from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
from datetime import timedelta as delta
from glob import glob
import numpy as np
import xarray as xr

ddir = '/data2/imau/oceanparcels/hydrodynamic_data/NEMO-MEDUSA/ORCA0083-N006/'
ufiles = sorted(glob(ddir+'means/ORCA0083-N06_*d05U.nc'))
vfiles = [u.replace('05U.nc', '05V.nc') for u in ufiles]
meshfile = glob(ddir+'domain/coordinates.nc')

filenames = {'U': {'lon': meshfile, 'lat': meshfile, 'data': ufiles},
             'V': {'lon': meshfile, 'lat': meshfile, 'data': vfiles}}
variables = {'U': 'uo', 'V': 'vo'}
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
fieldset = FieldSet.from_nemo(filenames, variables, dimensions)

galapagos_extent = [-91.8, -89, -1.4, 0.7]
startlon, startlat = np.meshgrid(np.arange(galapagos_extent[0], galapagos_extent[1], 0.2),
                                 np.arange(galapagos_extent[2], galapagos_extent[3], 0.2))

def Age(fieldset, particle, time):
    particle.age = particle.age + math.fabs(particle.dt)
    if particle.age > 10*365*86400:
        particle.delete()

class GalapagosParticle(JITParticle):
    age = Variable('age', initial = 0.)

pset = ParticleSet(fieldset=fieldset, pclass=GalapagosParticle, lon=startlon, lat=startlat, 
                   time=fieldset.U.grid.time[-1], repeatdt=delta(days=10))
outfile = pset.ParticleFile(name="/scratch/evansebille/galapagosparticles_bwd.nc", outputdt=delta(days=1))

pset.execute(AdvectionRK4+pset.Kernel(Age), dt=delta(hours=-1), output_file=outfile)

         It will be opened with no decoding. Filling values might be wrongly parsed.
/home/staff/sebil001/miniconda2/lib/python2.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable u'uo' has multiple fill values set([0.0, 1e+20]), decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/staff/sebil001/miniconda2/lib/python2.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable u'tauuo' has multiple fill values set([0.0, 1e+20]), decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/staff/sebil001/miniconda2/lib/python2.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable u'vo' has multiple fill values set([0.0, 1e+20]), decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/staff/sebil001/miniconda2/lib/python2.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable u'tauvo' has multiple fill values set([0.0, 1e+20]), decoding all values to NaN.
  stack_char_dim=stack_cha

In [3]:
import numpy as np
galapagos_extent = [-91.8, -89, -1.4, 0.7]
startlon, startlat = np.meshgrid(np.arange(galapagos_extent[0], galapagos_extent[1], 0.2),
                                 np.arange(galapagos_extent[2], galapagos_extent[3], 0.2))
startlon.shape

(11, 14)